### Sandbox for exporting camera metadata from a database

In [ ]:
from pathlib import Path

import polars as pl

from afft.io.sql import create_endpoint
from afft.utils.log import logger
from afft.utils.result import Ok, Err

from afft.tasks.process_tables import process_database_tables


def get_task_configurations() -> list[tuple]:
    """Return a set a task configurations."""
    
    queries: dict[str, dict] = dict()
    outputs: dict[str, Path] = dict()
    
    queries["2010"] = {
        "ctd": "SELECT * from r23685bc_20100605_021022_ctd_seabird",
        "dvl": "SELECT * from r23685bc_20100605_021022_dvl_teledyne",
        "image": "SELECT * from r23685bc_20100605_021022_image_capture",
        "pressure": "SELECT * from r23685bc_20100605_021022_pressure_parosci",
        "flourometer": "SELECT * from r23685bc_20100605_021022_flourometer_ecopuck",
    }

    queries["2012"] = {
        "ctd": "SELECT * from r23685bc_20120530_233021_ctd_seabird",
        "dvl": "SELECT * from r23685bc_20120530_233021_dvl_teledyne",
        "image": "SELECT * from r23685bc_20120530_233021_image_capture",
        "pressure": "SELECT * from r23685bc_20120530_233021_pressure_parosci",
        "flourometer": "SELECT * from r23685bc_20120530_233021_flourometer_ecopuck",
    }
    
    queries["2014"] = {
        "ctd": "SELECT * from r23685bc_20140616_225022_ctd_seabird",
        "dvl": "SELECT * from r23685bc_20140616_225022_dvl_teledyne",
        "image": "SELECT * from r23685bc_20140616_225022_image_capture",
        "pressure": "SELECT * from r23685bc_20140616_225022_pressure_parosci",
        "flourometer": "SELECT * from r23685bc_20140616_225022_flourometer_ecopuck",
    }

    outputs["2010"] = Path(
        "/data/kingston_snv_01/acfr_cameras_metadata/r23685bc_20100605_021022_camera_metadata.csv"
    )
    outputs["2012"] = Path(
        "/data/kingston_snv_01/acfr_cameras_metadata/r23685bc_20120530_233021_camera_metadata.csv"
    )
    outputs["2014"] = Path(
        "/data/kingston_snv_01/acfr_cameras_metadata/r23685bc_20140616_225022_camera_metadata.csv"
    )

    configurations: list[tuple] = [
        ( queries.get("2010"), outputs.get("2010") ),
        ( queries.get("2012"), outputs.get("2012") ),
        ( queries.get("2014"), outputs.get("2014") ),
    ]

    return configurations


def main() -> None:
    """Main function."""

    configurations: list[tuple[dict, Path]] = get_task_configurations()

    match create_endpoint(database="acfr_auv_messages", host="localhost", port=5432):
        case Ok(endpoint):
            for queries, output in configurations:
                processed: pl.DataFrame = process_database_tables(
                    endpoint, queries, base="image"
                )
                processed.write_csv(output)
                logger.info(f"Successfully wrote table to: {output}")
        case Err(message):
            logger.error(message)


# INVOKE MAIN
main()